## Install Required Packages

In [ ]:
# %pip install --upgrade raiwidgets
# %pip install --upgrade pandas
# %pip install --upgrade fairlearn
# %pip install --upgrade interpret-community

After installing packages, you must close and reopen the notebook as well as restarting the kernel.

## Counterfactuals and Causal Effects



**Counterfactual Analysis** powered by [DiCE](https://github.com/interpretml/DiCE), which shows feature-perturbed versions of the same datapoint who would have received a different prediction outcome, e.g., Taylor's loan has been rejected by the model. But they would have received the loan if their income was higher by $10,000.

**Causal Analysis** powered by [EconML](https://github.com/py-why/EconML), which focuses on answering What If-style questions to apply data-driven decision-making – how would revenue be affected if a corporation pursues a new pricing strategy? Would a new medication improve a patient’s condition, all else equal?


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from responsibleai import RAIInsights
from raiwidgets import ResponsibleAIDashboard

In [ ]:
# Load the dataset
data = pd.read_csv('housing_prices.csv')

# Check column names for verification
print("Column names in the dataset:", data.columns)

# Define target column
target_column = 'price'  # Replace 'price' with the correct column name if different

In [ ]:
# Ensure target column exists
if target_column in data.columns:
    print(f"Target column '{target_column}' exists.")
else:
    print(f"Target column '{target_column}' does NOT exist. Check column name.")

# Separate features (X) and target (y)
X = data.drop(columns=[target_column])  # Drop target column from features
y = data[target_column]  # Target column

# Convert the target column into DataFrame to match the expected input format
y = pd.DataFrame(y, columns=[target_column])

# Handle categorical variables by converting to numeric (one-hot encoding)
X = pd.get_dummies(X, drop_first=True)  # Convert categorical variables to one-hot encoding

# Ensure all feature columns are numeric
X = X.astype(float)

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train.values.ravel())  # Use .ravel() to make sure target is a 1D array

# Combine training and test features with their corresponding targets for RAIInsights
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

In [ ]:
# Create RAIInsights object
rai_insights = RAIInsights(model=model,
                           train=train_data,
                           test=test_data,
                           target_column=target_column,
                           task_type='regression')

# Generate counterfactuals with a broader desired range
# Adjust these values based on your data distribution
desired_range = [data[target_column].min(), data[target_column].max()]  # Use full range of target values
rai_insights.counterfactual.add(total_CFs=20, desired_range=desired_range)

# Check the treatment features
treatment_features = ['SquareFeet', 'Bedrooms', 'Bathrooms']  # Ensure these features are correct
rai_insights.causal.add(treatment_features=treatment_features)

In [ ]:
# Compute insights
rai_insights.compute()

In [ ]:
# Instantiate Responsible AI Dashboard
ResponsibleAIDashboard(rai_insights)

## © Copyright, 2025 Assentian Limited. All Rights Reserved 